In [1]:
# change dir to root
%cd ../

c:\Users\Anant\Work\search-app


## Evaluation

In [2]:
import json
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import ndcg_score, average_precision_score
from core import TFIDFSearch, BM25Search, NeuralSearch, HybridSearch, RETRIEVAL_MODELS, SCORE_TYPES, MEAN_TYPES, NORM_TYPES

In [3]:
def get_metrics(true_scores_df, model, mean_type=None, norm_type=None, k=20):
    ndcg = []
    pbar = tqdm(
        desc="Evaluating nDCG Score ",
        total=true_scores_df.shape[0],
        unit="query",
        leave=False,
    )
    for index, row in true_scores_df.iterrows():
        if model.__class__.__name__ == "HybridSearch":
            y_score = (
                model.get_scores(row.QUERY, mean_type, norm_type)
                .sort_index()
                .SCORE.tolist()
            )
        else:
            y_score = model.get_scores(row.QUERY).sort_index().SCORE.tolist()
        y_true = json.loads(row.SCORES)
        ndcg.append(ndcg_score([y_true], [y_score], k=k))
        pbar.update(1)
    pbar.close()

    return np.mean(ndcg)

In [4]:
true_scores = pd.read_csv('./data/processed/true_scores.csv')

### TFIDF

In [5]:
model = TFIDFSearch()
print('NDCG Score:',get_metrics(true_scores, model))

Evaluating nDCG Score :   0%|          | 0/3333 [00:00<?, ?query/s]

NDCG Score: 0.9114985854389379


### BM25

In [6]:
model = BM25Search()
print('NDCG Score:',get_metrics(true_scores, model))

Evaluating nDCG Score :   0%|          | 0/3333 [00:00<?, ?query/s]

NDCG Score: 0.9027729326623276


### Neural

In [ ]:
for model_name in RETRIEVAL_MODELS:
    print(f'Model: {model_name}')
    for score_type in SCORE_TYPES:
        model = NeuralSearch(model=model_name, score_type=score_type)
        score = get_metrics(true_scores, model)
        print(f'Score Type: {score_type}\t;\tScore: {score}')
        del(model)
        torch.cuda.empty_cache()
    print('\n')

In [ ]:
Model: BAAI/bge-base-en-v1.5
Score Type: Dot Product	        ;	Score: 0.8575061348159865
Score Type: Cosine Similarity	;	Score: 0.8575061348159865

Model: thenlper/gte-large
Score Type: Dot Product	        ;	Score: 0.8773796912454879
Score Type: Cosine Similarity	;	Score: 0.8773796912454879

Model: llmrails/ember-v1
Score Type: Dot Product	        ;	Score: 0.8041243430795993
Score Type: Cosine Similarity	;	Score: 0.8434121092569689

Model: thenlper/gte-base
Score Type: Dot Product	        ;	Score: 0.8843679159293903
Score Type: Cosine Similarity	;	Score: 0.8843679159293903

Model: all-distilroberta-v1
Score Type: Dot Product	        ;	Score: 0.7205834484750344
Score Type: Cosine Similarity	;	Score: 0.7205834484750344

Model: msmarco-distilbert-base-v4
Score Type: Dot Product	        ;	Score: 0.7505768344386088
Score Type: Cosine Similarity	;	Score: 0.7674091011409188

Model: msmarco-MiniLM-L-6-v3
Score Type: Dot Product	        ;	Score: 0.6871907026867569
Score Type: Cosine Similarity	;	Score: 0.7588424587892733

Model: msmarco-MiniLM-L-12-v3
Score Type: Dot Product	        ;	Score: 0.7376601596872528
Score Type: Cosine Similarity	;	Score: 0.7880065060269283

Model: msmarco-distilbert-base-tas-b
Score Type: Dot Product	        ;	Score: 0.9012300542971058
Score Type: Cosine Similarity	;	Score: 0.8716073130245685

### Hybid

In [ ]:
for model_name in RETRIEVAL_MODELS:
    print(f'Model: {model_name}')
    for mean_type in MEAN_TYPES:
        for norm_type in [None]+NORM_TYPES:
            for score_type in SCORE_TYPES:
                model = HybridSearch(model=model_name, score_type=score_type)
                score = get_metrics(true_scores, model, mean_type, norm_type)
                print(f'Mean Type: {mean_type}\t;\tNorm Type: {norm_type}\t;\tScore Type: {score_type}\t;\tScore: {score}')
                del(model)
                torch.cuda.empty_cache()
    print('\n')

In [ ]:
Model: BAAI/bge-base-en-v1.5
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9404823059231748
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9404823059231748
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9406323735097765
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9406323735097765
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9381730386226522
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9381730386226522
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.906591376131558
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.906591376131558
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.906591376131558
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.906591376131558
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.908403983772282
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.908403983772282
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.907573547288544
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.907573547288544
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9103763005702836
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9103763005702836
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9088665809755663
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9088665809755663


Model: thenlper/gte-large
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9416377062981929
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9416377062981929
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.941611383105613
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.941611383105613
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.945729762984324
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.945729762984324
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.906725652726001
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.906725652726001
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.906725652726001
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.906725652726001
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9128019346323036
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9128019346323036
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.912265174934058
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.912265174934058
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9123662269116276
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9123662269116276
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9129155237046395
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9129155237046395


Model: llmrails/ember-v1
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8433413830906041
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9374255311163692
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.935698322955202
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9374569783019548
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9322476561211566
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9375673025407658
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9068510484725307
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9093774021322129
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9068510484725307
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9093774021322129
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9078807884278359
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9114604815685324
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9036730358544254
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9124875116305171
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9095566002041273
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9136068565238556
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9088610025623405
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9117326598061203


Model: thenlper/gte-base
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9426274471521076
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9426274471521076
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9425341957478749
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9425341957478749
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9433059488344918
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9433059488344918
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9054415735302624
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9054415735302624
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9054415735302624
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9054415735302624
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9101136382871488
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9101136382871488
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9095688814063454
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9095688814063454
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9097522296731663
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9097522296731663
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9113946289110133
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9113946289110133


Model: msmarco-distilbert-base-v4
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8124524872914669
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9364812976973078
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9337995168681117
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9366872892992631
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.92949454934997
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9334576845950363
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9010649670560243
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9039424451227377
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9010649670560243
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9039424451227377
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9033684616680311
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.905506451809487
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9035758141006619
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.871862319090378
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.896939534762211
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.8997614110645613
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9039771502169843
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9056377540375286


Model: msmarco-MiniLM-L-6-v3
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8012344956982066
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9292199428123454
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9165696120226009
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9270649164747999
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9092715370097105
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9210028411041004
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8888500508685514
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.8968667985378287
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.8888500508685514
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.8968667985378287
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.8921684494681199
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.8992281374187605
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8982043595202135
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.8748191363957049
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.8797570158889051
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.8930827601016538
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.8917087718512503
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.8999012055136406


Model: msmarco-MiniLM-L-12-v3
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8776032245724033
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.933440650056653
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9285852069998198
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9358733174099106
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9207252898921064
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.932237276444628
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8991122679279592
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9073691143897434
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.8991122679279592
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9073691143897434
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9016830626720477
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9079785393293677
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.901939486796454
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.8885054530108357
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.8942753998659146
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9044254897352813
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9020679044067372
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9080186970029352


Model: msmarco-distilbert-base-tas-b
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9443515165066183
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9384146895631025
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9397112660289783
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9384199146705682
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9459423058906687
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9340897358688408
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9056872248380489
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9037895806836272
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9056872248380489
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9037895806836272
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9125942540571682
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9049612916413237
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.90351102668269
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9048335189372935
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9102277523281436
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9054845825893869
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9127082720837091
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9054176192830672


Model: all-distilroberta-v1
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.9319617353594265
Mean Type: Arithmetic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.9319617353594265
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.9274071349951848
Mean Type: Arithmetic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.9274071349951848
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.9164160498967063
Mean Type: Arithmetic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.9164160498967063
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8953593630530595
Mean Type: Geometric	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.8953593630530595
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.8953593630530595
Mean Type: Geometric	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.8953593630530595
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.895473953919277
Mean Type: Geometric	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.895473953919277
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Dot Product	;	Score: 0.8768468459413084
Mean Type: Harmonic	;	Norm Type: None	;	Score Type: Cosine Similarity	;	Score: 0.8768468459413084
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Dot Product	;	Score: 0.8907502820151039
Mean Type: Harmonic	;	Norm Type: L2	;	Score Type: Cosine Similarity	;	Score: 0.8907502820151039
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Dot Product	;	Score: 0.8961730701211849
Mean Type: Harmonic	;	Norm Type: Min-Max	;	Score Type: Cosine Similarity	;	Score: 0.8961730701211849